In [1]:
# 📦 1. Mediapipe 설치
!pip install mediapipe

# 📥 2. 모델 다운로드
!wget -q -O pose_landmarker.task https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task

# 📚 3. 라이브러리 import
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

import numpy as np
import cv2
import PIL.Image
import io
import time
from base64 import b64decode
from IPython.display import Javascript, display
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow

# 🎥 4. 웹캠 JavaScript 캡처 함수
def video_stream():
    js = Javascript('''
        async function startStream() {
            const video = document.createElement('video');
            const canvas = document.createElement('canvas');
            const ctx = canvas.getContext('2d');
            const stream = await navigator.mediaDevices.getUserMedia({ video: true });

            video.srcObject = stream;
            await video.play();

            document.body.appendChild(video);
            document.body.appendChild(canvas);
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;

            function captureFrame() {
                ctx.drawImage(video, 0, 0, canvas.width, canvas.height);
                return canvas.toDataURL('image/jpeg', 0.8);
            }

            return new Promise((resolve) => {
                setInterval(() => resolve(captureFrame()), 100);
            });
        }
    ''')
    display(js)
    return eval_js('startStream()')

# 📸 5. 프레임 가져오는 함수
def get_frame():
    frame_data = video_stream()
    binary = b64decode(frame_data.split(',')[1])
    image = PIL.Image.open(io.BytesIO(binary))
    image_np = np.array(image)
    return cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)

# 🤖 6. Mediapipe Pose 모델 초기화
base_options = python.BaseOptions(model_asset_path='pose_landmarker.task')
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=False,
    running_mode=vision.RunningMode.IMAGE
)
detector = vision.PoseLandmarker.create_from_options(options)

# 🧼 7. 웹캠 요소 제거 함수 (선택)
def remove_video_elements():
    js = Javascript('''
        function removeVideos() {
            let videos = document.querySelectorAll('video');
            let canvases = document.querySelectorAll('canvas');
            videos.forEach(video => video.remove());
            canvases.forEach(canvas => canvas.remove());
        }
        removeVideos();
    ''')
    display(js)


In [2]:
# 33개 랜드마크 연결 정의 (Mediapipe 공식 연결 순서 일부 예시)
POSE_CONNECTIONS = [
    (0, 1), (1, 2), (2, 3), (3, 7),     # 오른팔
    (0, 4), (4, 5), (5, 6), (6, 8),     # 왼팔
    (9, 10),                           # 어깨
    (11, 12), (11, 13), (13, 15),      # 왼쪽 다리
    (12, 14), (14, 16),                # 오른쪽 다리
    (11, 23), (12, 24),                # 허리 연결
    (23, 24), (23, 25), (24, 26),      # 골반-다리
    (25, 27), (26, 28),
    (27, 29), (28, 30),
    (29, 31), (30, 32)
]

def draw_landmarks_on_frame(frame, landmarks):
    annotated = frame.copy()
    h, w, _ = frame.shape

    # 점 그리기
    for lm in landmarks:
        x = int(lm.x * w)
        y = int(lm.y * h)
        cv2.circle(annotated, (x, y), 4, (0, 255, 0), -1)

    # 선 연결
    for start_idx, end_idx in POSE_CONNECTIONS:
        if start_idx < len(landmarks) and end_idx < len(landmarks):
            x0, y0 = int(landmarks[start_idx].x * w), int(landmarks[start_idx].y * h)
            x1, y1 = int(landmarks[end_idx].x * w), int(landmarks[end_idx].y * h)
            cv2.line(annotated, (x0, y0), (x1, y1), (0, 255, 0), 2)

    return annotated

In [5]:
import time
import numpy as np

fps = 10  # 초당 프레임 수
duration = 30  # 총 수집 시간 (초)
interval = 1 / fps  # 프레임 간 간격 = 0.1초
captured_frames = []

start_time = time.time()
last_capture = start_time

print(f"📸 {duration}초 동안 초당 {fps}프레임 수집 시작...")

while True:
    now = time.time()

    # 일정 간격마다만 프레임 캡처
    if now - last_capture >= interval:
        frame = get_frame()  # 👉 네가 이미 정의한 실시간 프레임 수집 함수
        captured_frames.append(frame)
        last_capture = now
        print(f"✅ {len(captured_frames)} 프레임 수집됨", end='\r')

    # 시간 다 지나면 종료
    if now - start_time >= duration:
        break

print(f"\n✅ 프레임 수집 완료! 총 {len(captured_frames)} 프레임")

📸 30초 동안 초당 10프레임 수집 시작...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ 37 프레임 수집됨
✅ 프레임 수집 완료! 총 37 프레임


In [6]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
landmark_list = []

for idx, frame in enumerate(captured_frames):
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        landmarks = [[lm.x, lm.y, lm.z] for lm in results.pose_landmarks.landmark]
        landmark_list.append(landmarks)
        print(f"✅ 프레임 {idx+1}: 포즈 추출 완료")
    else:
        print(f"⛔ 프레임 {idx+1}: 포즈 인식 실패")

# 저장
live_pose = np.array(landmark_list)
np.save("live_pose.npy", live_pose)
print(f"💾 좌표 저장 완료! 저장된 프레임 수: {len(live_pose)}")


⛔ 프레임 1: 포즈 인식 실패
⛔ 프레임 2: 포즈 인식 실패
⛔ 프레임 3: 포즈 인식 실패
⛔ 프레임 4: 포즈 인식 실패
⛔ 프레임 5: 포즈 인식 실패
⛔ 프레임 6: 포즈 인식 실패
⛔ 프레임 7: 포즈 인식 실패
⛔ 프레임 8: 포즈 인식 실패
⛔ 프레임 9: 포즈 인식 실패
⛔ 프레임 10: 포즈 인식 실패
⛔ 프레임 11: 포즈 인식 실패
⛔ 프레임 12: 포즈 인식 실패
⛔ 프레임 13: 포즈 인식 실패
⛔ 프레임 14: 포즈 인식 실패
⛔ 프레임 15: 포즈 인식 실패
⛔ 프레임 16: 포즈 인식 실패
✅ 프레임 17: 포즈 추출 완료
✅ 프레임 18: 포즈 추출 완료
✅ 프레임 19: 포즈 추출 완료
⛔ 프레임 20: 포즈 인식 실패
⛔ 프레임 21: 포즈 인식 실패
⛔ 프레임 22: 포즈 인식 실패
⛔ 프레임 23: 포즈 인식 실패
⛔ 프레임 24: 포즈 인식 실패
⛔ 프레임 25: 포즈 인식 실패
⛔ 프레임 26: 포즈 인식 실패
⛔ 프레임 27: 포즈 인식 실패
⛔ 프레임 28: 포즈 인식 실패
⛔ 프레임 29: 포즈 인식 실패
⛔ 프레임 30: 포즈 인식 실패
⛔ 프레임 31: 포즈 인식 실패
⛔ 프레임 32: 포즈 인식 실패
⛔ 프레임 33: 포즈 인식 실패
⛔ 프레임 34: 포즈 인식 실패
⛔ 프레임 35: 포즈 인식 실패
⛔ 프레임 36: 포즈 인식 실패
⛔ 프레임 37: 포즈 인식 실패
💾 좌표 저장 완료! 저장된 프레임 수: 3


In [7]:
live_pose = np.array(landmark_list)
np.save("live_pose.npy", live_pose)

In [9]:
# 📚 필요한 라이브러리
!pip install fastdtw

import numpy as np
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

# 파일 불러오기
reference = np.load("reference_pose.npy")
live = np.load("live_pose.npy")

# [프레임 수, 33, 3] → [프레임 수, 99] 평탄화
reference_flat = reference.reshape(reference.shape[0], -1)
live_flat = live.reshape(live.shape[0], -1)

# DTW 거리 계산
distance, _ = fastdtw(live_flat, reference_flat, dist=euclidean)

print(f"\n📏 FastDTW 거리: {distance:.2f}")

FileNotFoundError: [Errno 2] No such file or directory: 'reference_pose.npy'

In [10]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import mediapipe as mp

# 데이터 로드
live_pose = np.load("live_pose.npy")  # shape = [프레임 수, 33, 3]

# Mediapipe 연결 정보
mp_pose = mp.solutions.pose
connections = mp_pose.POSE_CONNECTIONS

fig, ax = plt.subplots(figsize=(5, 5))
points, = ax.plot([], [], 'ro', markersize=4)
lines = [ax.plot([], [], 'b-')[0] for _ in connections]

def init():
    ax.set_xlim(0, 1)
    ax.set_ylim(1, 0)
    ax.axis('off')
    return [points] + lines

def update(frame_idx):
    landmarks = live_pose[frame_idx]
    xs = [lm[0] for lm in landmarks]
    ys = [lm[1] for lm in landmarks]

    points.set_data(xs, ys)
    for i, (start, end) in enumerate(connections):
        x_line = [landmarks[start][0], landmarks[end][0]]
        y_line = [landmarks[start][1], landmarks[end][1]]
        lines[i].set_data(x_line, y_line)

    return [points] + lines

ani = animation.FuncAnimation(fig, update, frames=len(live_pose), init_func=init, interval=50, blit=True)
plt.close()

from IPython.display import HTML
HTML(ani.to_jshtml())

In [ ]:
import numpy as np

reference_pose = np.load("reference_pose.npy")
live_pose = np.load("live_pose.npy")

In [ ]:
# 왼쪽 무릎 landmark index = 25
ref_knees = [frame[25][1] for frame in reference_pose]
live_knees = [frame[25][1] for frame in live_pose]

lowest_ref_idx = np.argmin(ref_knees)
lowest_live_idx = np.argmin(live_knees)

ref_frame = reference_pose[lowest_ref_idx]
live_frame = live_pose[lowest_live_idx]

print(f"📍 기준자세 프레임 idx: {lowest_ref_idx}, 실시간 프레임 idx: {lowest_live_idx}")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import mediapipe as mp

# 데이터 불러오기
reference = np.load("reference_pose.npy")
live = np.load("live_pose.npy")

# 프레임 수 맞추기 (짧은 쪽에 맞춤)
min_frames = min(len(reference), len(live))
reference = reference[:min_frames]
live = live[:min_frames]

# 관절 연결 정보
mp_pose = mp.solutions.pose
connections = mp_pose.POSE_CONNECTIONS

# 시각화 세팅
fig, ax = plt.subplots(figsize=(6, 6))

def update(frame_idx):
    ax.clear()
    ax.set_xlim(0, 1)
    ax.set_ylim(1, 0)
    ax.set_title(f"Frame {frame_idx}: 🟢 기준 vs 🔵 내 자세")
    ax.axis('off')

    ref = reference[frame_idx]
    live_f = live[frame_idx]

    # 기준 자세 (초록)
    xs1 = [pt[0] for pt in ref]
    ys1 = [pt[1] for pt in ref]
    ax.scatter(xs1, ys1, c='green', s=10)
    for s, e in connections:
        ax.plot([xs1[s], xs1[e]], [ys1[s], ys1[e]], 'g-', alpha=0.5)

    # 내 자세 (파랑)
    xs2 = [pt[0] for pt in live_f]
    ys2 = [pt[1] for pt in live_f]
    ax.scatter(xs2, ys2, c='blue', s=10)
    for s, e in connections:
        ax.plot([xs2[s], xs2[e]], [ys2[s], ys2[e]], 'b-', alpha=0.5)

ani = animation.FuncAnimation(fig, update, frames=min_frames, interval=100)
plt.close()

from IPython.display import HTML
HTML(ani.to_jshtml())